In [1]:
%cd /mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/

/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5


In [2]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
import warnings
warnings.filterwarnings("ignore")

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu102 (GeForce RTX 2060)


# Inference

In [3]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path = '/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/runs/train/exp3/weights/best.pt')  # or yolov5m, yolov5l, yolov5x, custom

Using cache found in /home/rajarshi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-6-5 Python-3.8.5 torch-1.11.0+cu102 CUDA:0 (GeForce RTX 2060, 5931MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
results = model('/mnt/036b1314-13ad-429f-b9f9-07467946c5d7/Datasets/Health/Leukomia/ALL-IDB/train/Im002_1.jpg')
coordinates = results.xyxy[0].cpu().numpy()
coordinates

array([[     627.26,      1167.7,      690.85,      1243.5,     0.46492,           0],
       [     1532.2,      508.91,        1597,      572.23,     0.43183,           1],
       [     757.38,      857.87,      817.84,      928.15,     0.40166,           0],
       [     846.74,       977.6,      911.72,      1039.2,     0.37993,           1],
       [     1545.4,      495.25,      1598.7,      551.56,     0.36927,           1],
       [     843.32,      999.68,      905.22,      1064.6,     0.34807,           1],
       [     1545.9,      534.74,      1600.6,      590.53,     0.33886,           1],
       [     850.84,      948.85,      910.18,      1011.4,     0.32702,           1],
       [     708.19,      701.14,       773.9,       766.4,     0.30429,           1],
       [     1133.9,      985.31,      1207.2,      1062.1,     0.26561,           0],
       [     1526.2,      533.32,      1578.9,      592.95,      0.2622,           1],
       [     573.32,      839.31,       627

In [ ]:
data_root_dir = '/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/JRF/Rat-Brain-Images-Segmentation'

group = 'Test'
folder = 'labels'
raw_img_dir = os.path.join(data_root_dir, 'images', group)
output_dir = os.path.join(data_root_dir, 'output', group, folder)

In [ ]:
# YOLO to COCO Conversion
def yolo_coord(i):
    image_width = 1200
    image_height = 900
    x_coco = i[1]
    y_coco = i[2]
    width_coco = i[3]
    height_coco = i[4]
    class_no = 0
    
    
    x_yolo = (x_coco + width_coco)/2
    y_yolo = (y_coco + height_coco)/2
    
    if x_coco > width_coco:
        width_yolo = x_coco - width_coco
    else:
        width_yolo = width_coco - x_coco
        
    if y_coco > height_coco:
        height_yolo = y_coco - height_coco
    else:
        height_yolo = height_coco - y_coco
    
    return [class_no, x_yolo, y_yolo, width_yolo, height_yolo] 

In [ ]:
# Model
wts_path = '/mnt/87433d53-66f3-438e-a7f4-fc990fe61283/PROJECTS/yolov5/runs/train/exp3/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path = wts_path)

for filename in os.listdir(raw_img_dir):
    file = os.path.join(raw_img_dir, filename)
    
    results = model(file)

    coordinates = results.xyxy[0].cpu().numpy()
    final_lis_coco = []
    for i in coordinates:
        lis = []
        for j in i:
            lis.append(j)
        lis.insert(0, lis.pop())
        lis.remove(lis[-1])

        final_lis_coco.append(lis) 
    
    final_lis_yolo = []
    for i in final_lis_coco:
        final_lis_yolo.append(yolo_coord(i))
        
    txt = open(os.path.join(output_dir, f'{filename[:-4]}.txt'), 'w+')
    for j in final_lis_yolo:
        txt.write(str(j) + "\n")
    txt.close()
    
    txt = open(os.path.join(output_dir, f'{filename[:-4]}.txt'), 'r')
    txt_read = txt.read()
    file = txt_read.replace('[', '').replace(']', '').replace(',', ' ')
    txt = open(os.path.join(output_dir, f'{filename[:-4]}.txt'), 'w+')
    txt.write(file)
    txt.close()